In [ ]:
import h5py
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, find_peaks
import numpy as np
import datetime
import pandas as pd
import obspy
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
import time
import glob
import os
import pytz
from tqdm import tqdm
import csv
import matplotlib.dates as mdates
import csv
import re



In [ ]:
#Loading template swarm 2023-08-27
file_list1 = glob.glob("/data/fast1/veronica-scratch-rainier-downsampling/drive1_ds/new_decimator_2023-08-27*")


file_list = file_list1 #+ file_list2 + file_list3 + file_list4 + file_list5 + file_list6 + file_list7 
file_list.sort()



In [ ]:
#parameters
chan_min = 1000
chan_max = 2500
#chan_min = 0
#chan_max = -1
channel_number = chan_max -chan_min
low_cut1 = 2
hi_cut1 = 9.0
#fs=attrs['MaximumFrequency']*2
fs = 20
samples_per_file = 60*fs

number_of_template = 1

# Buiding Template

In [ ]:
#first version, just one event


# Ruta del archivo HDF5
#template1

template_path = '/data/fast1/veronica-scratch-rainier-downsampling/drive1_ds/new_decimator_2023-08-27_12.37.00_UTC.h5'

#template2 
#rawdata_path = '/data/data4/veronica-scratch-rainier/drive1_ds/new_decimator_2023-08-27_11.38.00_UTC.h5'


# Abrir el archivo HDF5
with h5py.File(template_path, 'r') as rawdata_file:
    # Acceder a los datos que deseas
    rawdata = rawdata_file['Acquisition/Raw[0]/RawData']

    # convert the template in a numpy array
    this_template = np.array(rawdata[:, chan_min:chan_max])
    #closing file
    rawdata_file.close()
    
# filtering
b, a = butter(2, (low_cut1, hi_cut1), 'bp', fs=fs)
template_filt = filtfilt(b, a, this_template, axis=0)



In [ ]:
#Folder for the figures

# Extract the part of the filename containing the date and time
filename = os.path.basename(template_path)
date_time_match = re.search(r'(\d{4}-\d{2}-\d{2}_\d{2}\.\d{2}\.\d{2})', filename)
if date_time_match:
    date_time = date_time_match.group(1)
else:
    raise ValueError("No date and time found in the filename.")

# Separate date and time
date = date_time[:10]
time_minutes = date_time[11:16].replace('.', '-')

# Get the base directory
base_directory = '/home/velgueta/notebooks/RainierDas/Plot-Rainier-TM'

# Full path of the folder to create
folder_path = os.path.join(base_directory, f"{date}_{time_minutes}")

# Check if the folder already exists, if not, create it
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print(f"Folder '{date}_{time_minutes}' created in '{base_directory}'")
else:
    print(f"Folder '{date}_{time_minutes}' already exists in '{base_directory}'")


In [ ]:
#cutting file of template

#template2, template = template_filt[120:140,:]
template = template_filt[30:50,:]
plt.imshow(template.T, cmap='seismic',aspect='auto',vmin=-0.05,vmax=0.05)
plt.show()



In [ ]:
def correlate(s1,s2,mode="same"):

    # throw an error of input sizes are inconsistent
    if s1.shape != s2.shape:
        raise ValueError("s1 and s2 must have the same size!")

    # get fft size
    sz = s1.shape[0]
    n_bits = 1+int(np.log2(2*sz-1))
    fft_sz = 2**n_bits

    # take FFT along time axis for both
    fft_s1 = np.fft.fft(s1, fft_sz, axis=0)
    fft_s2 = np.fft.fft(s2, fft_sz, axis=0)

    # take complex conjugate of second signal
    fft_s2_conj = np.conj(fft_s2)

    # multiply to get correlation function
    corr_fft = fft_s1*fft_s2_conj

    # take inverse fourier transform
    corr = np.fft.ifft(corr_fft, axis=0)

    # normalize using the magnitude of both input data
    norm1 = np.linalg.norm(s1,axis=0)
    norm2 = np.linalg.norm(s2,axis=0)
    norm_factor = norm1*norm2
    corr = np.vstack((corr[-(sz-1) :], corr[:sz]))
    norm_corr = np.real(corr) / norm_factor

    # return desired part of correlation function
    if mode == "full":
        pass
    elif mode == "same":
        norm_corr = norm_corr[int(sz/2):-int(sz/2)+1]
    return norm_corr


def window_and_correlate(template,data):

    # define container
    all_corr = []

    # get some helpful values
    window_length = template.shape[0]
    num_windows = int(data.shape[0]/window_length)

    # iterate through time windows
    for i in range(num_windows):

        # pull out a time window of data
        start_index = i*window_length
        end_index = start_index + window_length
        window = data[start_index:end_index,:]

        # call cross correlation function
        corr = correlate(template,window)

        # save value
        all_corr.append(corr)

    # reshape output
    all_corr = np.stack(all_corr)

    return all_corr


In [ ]:
#Creating directories for each experiments

corrs = "corrs_templatetest"
freqs = "freqs-2-9"


daystart = file_list[0]
dayend = file_list[-1]

# Extract date, hour, and minute portion from file path, excluding "new" and "decimator"
daystart_parts = os.path.basename(daystart).split('_')
daystart_date_time = "_".join(daystart_parts[2:4])  # Extract the date and time parts, excluding "new_decimator" and "UTC.h5"

dayend_parts = os.path.basename(dayend).split('_')
dayend_date_time = "_".join(dayend_parts[2:4])  # Extract the date and time parts, excluding "new_decimator" and "UTC.h5"

# Base directory to save files
base_directory = '/data/data4/veronica-scratch-rainier/npyfiles/'

# Create the output folder name
folder_output = os.path.join(base_directory, f"{corrs}_{freqs}_{chan_min}_{chan_max}_{daystart_date_time}_{dayend_date_time}")

# Create the output folder if it doesn't exist
if not os.path.exists(folder_output):
    os.mkdir(folder_output)
    print(f"Directory '{folder_output}' created successfully.")
else:
    print(f"Directory '{folder_output}' already exists.")


In [ ]:
#no funciono la corrcorrelacion?? parece que si 


inicio_tiempo = time.perf_counter()


datetimestotal = []
corrs3_list = []
all_timestamps = []

for i, file in tqdm(enumerate(file_list)):
    with h5py.File(file, "r") as f:
        try:
            # Lee los datos del archivo y realiza las operaciones necesarias
            raw_data = np.array(f['Acquisition/Raw[0]/RawData'][:, chan_min:chan_max])
            timestamps = np.array(f['Acquisition/Raw[0]/RawDataTime'])  # tiempo en microsegundos desde 1970-01-01
            data_filt = filtfilt(b, a, raw_data, axis=0)

            all_timestamps.extend(timestamps)

            # Ajusta la longitud de los datos al número samples_per_file si es necesario
            if data_filt.shape != template.shape:
                data_filt = data_filt[:samples_per_file, :]

            # Calcula correlaciones incluso si la longitud no coincide
            corrs = window_and_correlate(template, data_filt)
            corrs2 = corrs.reshape((int(samples_per_file), channel_number))
            corrs3 = np.sum(corrs2, axis=1) / channel_number
            corrs3_list.append(corrs3)

        except Exception as e:
            print(f"Error al procesar el archivo {file}: {e}. Se omite este archivo.")
            continue
        finally:
            f.close()  
            # Asegura cerrar el archivo incluso si hay un error
            output_folder = folder_output   
     
        
        for i, corrs3 in enumerate(corrs3_list):
            outfile_name = os.path.join(output_folder, f'corrs3_{i}.npy')
            np.save(outfile_name, corrs3)
          
                    
# Registra el tiempo de finalización
fin_tiempo = time.perf_counter()

# Calcula el tiempo de ejecución
finaltime = fin_tiempo - inicio_tiempo

print(f"code took {finaltime} seconds.")




# graficando los archivos corrs3.npy

In [ ]:

#time steps
timesteps = len(file_list)* samples_per_file
# Extraer las fechas de inicio y fin del nombre de la carpeta de salida
folder_parts = os.path.basename(folder_output).split('_')
start_datetime = "_".join(folder_parts[-4:-2])  # Extract date and time from the second-to-last and last elements
end_datetime = "_".join(folder_parts[-2:])  # Extract date and time from the last two elements

# Convertir las cadenas de tiempo a objetos datetime
start_datetime = pd.to_datetime(start_datetime, format="%Y-%m-%d_%H.%M.%S")
end_datetime = pd.to_datetime(end_datetime, format="%Y-%m-%d_%H.%M.%S")

# Crear un rango de fechas con 1728000 puntos
time_range = pd.date_range(start=start_datetime, end=end_datetime, periods=timesteps)


# Juntar output_folder con folder_output
output_folder = folder_output

# Lista para almacenar las correlaciones cargadas de los archivos
corrs3_list = []

# Cargar los archivos npy y almacenar las correlaciones en corrs3_list
for i, file in enumerate(os.listdir(output_folder)):
    if file.startswith("corrs3_") and file.endswith(".npy"):
        filepath = os.path.join(output_folder, file)
        corrs3 = np.load(filepath)
        corrs3_list.append(corrs3)

# Unir todos los arreglos de correlaciones en uno solo
correlaciones_totales = np.concatenate(corrs3_list)

# Graficar las correlaciones en función del tiempo
plt.figure(figsize=(10, 6))
plt.plot(time_range, correlaciones_totales)
plt.xlabel('Time (UTC)')
plt.ylabel('Correlation Value')
plt.title('Total Correlation Results')
plt.show()


In [ ]:
#convert timestam to datetime

import datetime
# Convertir las marcas de tiempo en objetos datetime
datetime_objects = [datetime.datetime.fromtimestamp(ts / 1e6) for ts in all_timestamps]



In [ ]:
len(datetime_objects)*7

In [ ]:
# Crear una lista que contenga todos los puntos de correlación
corrs_all = []
for i, corrs3 in enumerate(corrs3_list):
    corrs_all.extend(corrs3)


corrs_all = np.array(corrs_all)
datetimes_all = np.array(datetime_objects)

# Supongamos que tienes un arreglo datetimes_array con 360,000 elementos
# y un arreglo corrs3_list que contiene 60 arreglos de 6000 puntos cada uno

# Definir la zona horaria original (supongamos que es Seattle)
zona_horaria_original = pytz.timezone('America/Los_Angeles')

# Convertir datetimes_array a UTC time
utc_timezone = pytz.timezone('UTC')
dates_times_utc = [date_time_pacific.astimezone(utc_timezone) for date_time_pacific in datetimes_all]


# Unir todos los arreglos de correlaciones en uno solo
correlaciones_totales = np.concatenate(corrs3_list)


In [ ]:
dates_times_utc

In [ ]:
#making nice plot

plt.figure(figsize=(20, 8))
plt.plot(dates_times_utc[0:len(correlaciones_totales)], correlaciones_totales[0:len(dates_times_utc)], linestyle='-', color='k', linewidth=1)
plt.ylim((0, 0.2))
#plt.xlim((, 90))
plt.xlabel("Time (UTC) since 27/8",fontsize = 20)
plt.ylabel("Cross correlation value",fontsize = 20)
plt.title(" Template using Template2 event_downsampling", fontsize = 20)

plt.grid(True)

# Personalizar los marcadores del eje x
plt.xticks(rotation=45)  # Rotar las etiquetas del eje x en 45 grados
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%H:%M:%S"))  # Formato de fecha y hora

# Agregar líneas verticales en cada elemento de la lista horas_datetime
#for hour_datetime in horas_datetime:
##
    #plt.axvline(x=hour_datetime, color='orange', linestyle='--')
    #plt.text(horas_datetime[10], 0.15, 'Quakes detected by PNSN', color='orange', rotation=0, fontsize=15, verticalalignment='bottom')

#plt.savefig('Mount-Rainier_5daytocompare_downsampling_1a98.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#testing method for define a thresold
thresholds = [0.5, 0.1, 0.05, 0.03]

# Iterate over each threshold
for threshold in thresholds:
    # Find indices where values exceed the threshold
    indices_above_threshold = np.where(np.abs(correlaciones_totales) > threshold)[0]

    # Find differences between consecutive indices
    diff_indices = np.diff(indices_above_threshold)

    # Find changes in differences indicating a new group
    group_changes = np.where(diff_indices > 40)[0]

    # Group indices into contiguous lists
    detection_groups = np.split(indices_above_threshold, group_changes + 1)

    # Count the number of detections for the current threshold
    count_above_threshold = len(detection_groups)

    # Print the result for the current threshold
    print(f"Detections for threshold downsampled {threshold}: {count_above_threshold}")


# testing method for define a thresold

In [ ]:
#poniendole el nombre del archivo txt, con el template que se uso.

# Rawdata file path
#rawdata_path = '/data/data4/veronica-scratch-rainier/drive1_ds/new_decimator_2023-08-27_10.10.00_UTC.h5'

# Extracting date from the file name using regular expressions
match = re.search(r'new_decimator_(\d{4}-\d{2}-\d{2}_\d{2}.\d{2}.\d{2})', template_path)
if match:
    date_with_hm = match.group(1)
    print("Date extracted from the file name:", date_with_hm)

    # Set the output folder for text files
    output_folder = 'txtfiles'
    os.makedirs(output_folder, exist_ok=True)

    # Text file name based on the extracted date
    output_file_path_TM = os.path.join(output_folder, f'results-TM_{date_with_hm}.txt')
    print("Text file name:", output_file_path_TM)


# Creating the txt files saving thresolds, numbers of dedections and dates

In [ ]:
import datetime

thresholds = [0.5, 0.1, 0.05, 0.04, 0.035, 0.03]  # Example thresholds
with open(output_file_path_TM, 'w') as file:
    # Write headers
    file.write("Threshold\tNumber of Detections\tDetection Times (UTC)\n")

    # Iterate over results for each threshold
    for threshold in thresholds:
        indices_above_threshold = np.where(np.abs(correlaciones_totales) > threshold)[0]
        diff_indices = np.diff(indices_above_threshold)
        group_changes = np.where(diff_indices > 20)[0]
        detection_groups = np.split(indices_above_threshold, group_changes + 1)

        # Iterate over each detection group
        for group in detection_groups:
            # Tomar solo la primera fecha en cada grupo
            first_detection_time_utc = datetime.datetime.utcfromtimestamp(dates_times_utc[group[0]].timestamp()).strftime('%Y-%m-%d_%H.%M')
            #first_detection_time_utc = datetime.datetime.utcfromtimestamp(dates_times_utc[group[0]].timestamp()).strftime('%Y-%m-%d_%H.%M.%S')
            # Escribir los datos en una fila separados por tabulaciones
            file.write(f"{threshold}\t{len(detection_groups)}\t{first_detection_time_utc}\n")

print(f"The results have been saved in {output_file_path_TM}")

# Making a nice plot of thrsolds versus detections.

In [ ]:
# Ruta al archivo de resultados
results_file_path = output_file_path_TM

# Listas para almacenar los datos
thresholds = []
num_detections = []

# Cargar datos desde el archivo results.txt
with open(results_file_path, 'r') as file:
    # Leer las líneas del archivo
    lines = file.readlines()

    # Iterar sobre las líneas, omitiendo la primera que son los encabezados
    for line in lines[1:]:
        # Dividir la línea en columnas
        columns = line.strip().split('\t')

        # Obtener umbral y número de detecciones
        threshold = float(columns[0])
        detections = int(columns[1])

        # Agregar datos a las listas
        thresholds.append(threshold)
        num_detections.append(detections)

# Configuración de estilo del gráfico
plt.style.use('seaborn-whitegrid')
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.viridis(np.linspace(0, 1, 12)))

# Graficar umbral versus número de detecciones

plt.figure(figsize=(10, 6))

# Graficar umbral versus número de detecciones
plt.scatter(thresholds, num_detections, marker='o', label='Data Points')
plt.xlabel('Threshold')
plt.ylabel('Number of Detections')
plt.title('Threshold vs Number of Detections')

# Ajustar los ejes automáticamente para mostrar el gráfico completo
plt.autoscale()


plt.xscale('linear')  # Escala lineal en el eje x

plt.legend()
plt.grid(True)
plt.show()


# line that compare the results_xxx.txt and the usgs txt made by me (Vero)

In [ ]:
from datetime import datetime, timedelta
import csv

In [ ]:


# Define the function to parse a date string into a datetime object
def parse_date(date_str):
    try:
        return datetime.strptime(date_str, '%Y-%m-%d_%H.%M.%S')
    except ValueError:
        return None

# Define the time difference allowed (5 seconds)
time_difference = timedelta(seconds=6)

# Function to read dates and values from a file and return a list of tuples containing both
def read_dates_and_values_from_file(file_path):
    dates_and_values = []
    with open(file_path, 'r') as f:
        next(f)  # Skip header line
        for line in f:
            elements = line.strip().split('\t')
            if len(elements) >= 2:  # Ensure there are at least two elements
                value = elements[0]  # Extract the value from the first column
                date_str = elements[-1]  # Extract the date from the last column
                date = parse_date(date_str)
                if date is not None:
                    dates_and_values.append((value, date))
    return dates_and_values

# Read dates and values from the converted_dates_and_values_rainier_5days.txt file
converted_dates_and_values = read_dates_and_values_from_file('converted_dates_and_values_rainier_5days.txt')

# Initialize lists to store common dates and unmatched dates
common_dates = []
unmatched_dates = []



# New output filename
output_filename = f"match_for_{date_with_hm}.txt"
output_file_path = os.path.join('txtfiles', output_filename)

# Read dates from the results file and compare with converted dates
with open(output_file_path_TM, 'r') as f:
    next(f)  # Skip header line
    for line in f:
        columns = line.strip().split('\t')
        if len(columns) >= 3:  # Ensure there are at least three elements in the list
            threshold_value = columns[0]  # Extract threshold value from the first column
            detection_times = columns[-1].split(',')  # Extract detection times from the last column
            for detection_time in detection_times:
                result_date = parse_date(detection_time)  # Convert detection time to a datetime object
                if result_date is not None:  # Check if result_date is not None
                    # Check if the result_date has a match in converted_dates_and_values
                    matched = False
                    for value, converted_date in converted_dates_and_values:
                        if abs(result_date - converted_date) <= time_difference:
                            common_dates.append((threshold_value, result_date, value))
                            matched = True
                            break
                    if not matched:
                        unmatched_dates.append((threshold_value, result_date))

# Write common and unmatched dates to the new output file in the 'txtfiles' directory

with open(output_file_path, 'w') as f:
    f.write("Threshold\tResult Date\tID Value\tMatch Status\n")
    for threshold_value, result_date, id_value in common_dates:
        f.write(f"{threshold_value}\t{result_date.strftime('%Y-%m-%d_%H.%M.%S')}\t{id_value}\tUGSGcatalog\n")
    for threshold_value, result_date in unmatched_dates:
        f.write(f"{threshold_value}\t{result_date.strftime('%Y-%m-%d_%H.%M.%S')}\tX\tUnmatched\n")

print(f"Output file '{output_file_path}' has been generated.")


In [ ]:


# Define paths and folders
txt_file_path = output_file_path
files_folder_path = '/data/data4/veronica-scratch-rainier/drive1_ds'
plots_folder = 'Plot-Rainier-TM'

# Regular expression pattern to search for dates in the correct format
date_pattern = re.compile(r'(\d{4}-\d{2}-\d{2})_(\d{2})\.(\d{2})')

# Extract information from output.txt
result_dates = []
thresholds = []
id_values = []
match_statuses = []

with open(output_file_path, 'r') as txt_file:
    for line in txt_file:
        columns = line.strip().split('\t')
        threshold = columns[0]
        result_date = columns[1]
        id_value = columns[2]
        match_status = columns[3]
        result_dates.append(result_date)
        thresholds.append(threshold)
        id_values.append(id_value)
        match_statuses.append(match_status)

# Define the threshold to search
threshold_to_search = '0.04'

# Create the folder for plots if it doesn't exist
if not os.path.exists(plots_folder):
    os.makedirs(plots_folder)

# Create a folder to hold all the plots
all_plots_folder = os.path.join(plots_folder, f'{threshold_to_search}_{date_with_hm}')
if not os.path.exists(all_plots_folder):
    os.makedirs(all_plots_folder)

# Iterate over the result dates and search for corresponding files with the specific threshold
for i, result_date in enumerate(result_dates):
    threshold = thresholds[i]
    id_value = id_values[i]
    match_status = match_statuses[i]
    match = date_pattern.match(result_date)
    if match:
        date_part = match.group(1)
        hour_part = match.group(2)
        minute_part = match.group(3)
        date_to_search = f"{date_part}_{hour_part}.{minute_part}"
        if threshold == threshold_to_search:
            found = False
            for file_name in os.listdir(files_folder_path):
                if date_to_search in file_name:
                    print(f"Found a file with the searched date: {file_name}")
                    found = True
                    
                    # Process the found file
                    chan_min =0
                    chan_max = -1
                    data_file_path = os.path.join(files_folder_path, file_name)
                    data_file = h5py.File(data_file_path, 'r')
                    this_data = np.array(data_file['Acquisition/Raw[0]/RawData'][:, chan_min:chan_max])
                    this_time = np.array(data_file['Acquisition/Raw[0]/RawDataTime'])
                    attrs = dict(data_file['Acquisition'].attrs)
                    data_file.close()

                    low_cut1 = 2
                    hi_cut1 = 9
                    fs = 20
                    b, a = butter(2, (low_cut1, hi_cut1), 'bp', fs=fs)
                    data_filt = filtfilt(b, a, this_data, axis=0)

                    date_format = mdates.DateFormatter('%H:%M:%S')
                    x_lims = mdates.date2num(this_time)
                    x_max = data_filt.shape[1] * attrs['SpatialSamplingInterval'] / 1000
                    dx = x_max / data_filt.shape[1]

                    fig, ax = plt.subplots(figsize=(20,10))
                    plt.imshow(data_filt.T, cmap='seismic', aspect='auto', vmin=-0.05, vmax=0.05, extent=[x_lims[0], x_lims[-1], x_max, 0])
                    plt.xlabel("Time UTC", fontsize=25)
                    plt.ylabel("Optical distance (km)", fontsize=25)
                    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
                    ax.xaxis_date()
                
                    # Set font size for time and channel axis labels
                    plt.xticks(fontsize=20)
                    plt.yticks(fontsize=20)
                    
                    # Set title including threshold, ID value, and match status
                    plt.title(f'Threshold: {threshold}, ID: {id_value}, Match Status: {match_status}', fontsize=20)
                    
                    # Save the plot in the all_plots_folder
                    full_path = os.path.join(all_plots_folder, f'{file_name}.png')
                    plt.savefig(full_path, dpi=300, bbox_inches='tight')
                    plt.close(fig)
                    break

            if not found:
                print(f"No matching file found for the searched date: {date_to_search}")


In [ ]:
# Path to the folder containing the saved plots

#plots_folder = f"Plot-Rainier-TM/{threshold_to_search}_{date_with_hm}"
#plots_folder = "/data/data4/veronica-scratch-rainier/plot-TM-results/0.04_2023-08-27_09.58.00"
#plots_folder = "Plot-Rainier-TM"
# Iterate over the saved plots and display them
#for file_name in os.listdir(plots_folder):
#    if file_name.endswith('.png'):
#        # Open the image file
#        image_path = os.path.join(plots_folder, file_name)
#        image = plt.imread(image_path)
        
        # Display the image
#        plt.figure(figsize=(10, 6))
#        plt.imshow(image)
#        plt.title(file_name)
#        plt.axis('off')  # Turn off axis labels
#        plt.show()